# 패키지 설명
- DictCollector(DictionaryCollect) : 프로파일링 대상 DB에 접속하여 스키마 정보 수집 관련 메소드
- SQLCreator(SQLCreate) : 수집된 스키마 정보를 기반으로 프로파일링 SQL을 생성하는 메소드
- Executer(ProfilingExecute) : 프로파일링 SQL 수행시키고, 프로파일링 결과를 수집하는 메소드

### 1. 패키지 임포트 및 선언

In [ ]:
import pyDataProfiling
DataProfiler = pyDataProfiling.DataProfiling()

### 2. 리포지터리DB 지정 및 접속
1) 리포지터리DB : 프로파일링 수행 결과와 SQL, 서버정보, 스키마정보(테이블, 컬럼) 등을 관리함 <br>
2) 리포지터리DB 테이블 정보는 별도의 데이터모델 파일 참조 <br>

In [ ]:
### 리포지터리DB 지정
DataProfiler.RepositoryDBSetting(dbtype='Tibero', ip='127.0.0.1', port=1521, dbname='trcl', userid='user', password='password')

In [ ]:
### 리포지터리DB 접속
DataProfiler.RepositoryDBConnect()

### 3. 프로파일링 대상 DB 정의 및 접속
1) 프로파일링을 수행할 데이터베이스를 정의할 수 있다.  
2) 셋팅 : 프로파일링 대상DB를 하나씩 지정이 가능하며, DB번호 기준으로 리포지터리DB에 Overwrite 된다.  
3) 로드 : 리포지터리DB에 저장된 DB정보를 일괄 호출한다.  
4) 접속 : 지정된 프로파일링 대상 DB에 접속을 시도한다.

In [ ]:
### 셋팅
DataProfiler.ProfilingDBSetting(DB번호=99, DB명='테스트', dbtype='Tibero', ip='127.0.0.1', port=1521, dbname='trcl', userid='user', password='password')

In [ ]:
### 로드
DataProfiler.ProfilingDBLoad()

In [ ]:
### 접속
DataProfiler.ProfilingDBConnect(DB번호=99)

### 4. 스키마정보 수집 (수집 변수 설명)

1) return_type : 스키마 정보 수집된 결과를 어떻게 리턴 받을지 정함 
 - Repository : 리포지터리DB에 바로 적재함, 적재되는 DB의 성능에 따라 느릴 수 있음
 - DataFrame : pandas DataFrame 형태로 리턴됨, 여러 DB를 최대한 한 시점으로 받고 싶을 경우 실행하여, dump형식으로 내릴때 유용함  
   
2) ignore : return_type이 'Repository' 일때만 사용하여, 이미 데이터가 존재할 경우 무시(※이전 데이터 삭제)할지를 정함
 - True : 이전 데이터 존재시 삭제(DB번호 기준)
 - False : 이전 데이터 존재시 오류 발생시킴

In [ ]:
DataProfiler.DictCollector.Collect(return_type='Repository', ignore=False)

In [ ]:
DataProfiler.DictCollector.Collect(return_type='Repository', ignore=True)

In [ ]:
data = DataProfiler.DictCollector.Collect(return_type='DataFrame')

### 5. SQL생성
### 프로파일링을 수행할 SQL을 생성하며, 리포지터리DB에만 접속하여 사용 가능
1) 딕셔너리 조회(DictionaryListSelect) : SQL을 생성할 DB번호를 지정하며, schemaList에 지정된 목록만 스키마를 생성함  
2) ColumnProfingSQLCreate : 컬럼의 기초통계치를 수집  
3) SampleProfingSQLCreate : 컬럼별로 샘플 데이터를 수집함 (최대 10건으로 고정)  
4) ExecuteErrorSQLCreate : 오류가 발생한 SQL을 컬럼 하나씩 수집할 수 있도록 변경함

- estimate : full 또는 sample로 지정  
- rowcount : estimate가 sample일 경우, 수집하고자 하는 최대 건수 지정  
- columncount : 한 SQL당 수집되는 컬럼 개수를 지정  

In [ ]:
### 딕셔너리 조회
DataProfiler.SQLCreator.DictionaryListSelect(DB번호=99, schemaList=['DTWAREADM'])

In [ ]:
### 컬럼기초통계치 수집 SQL 생성
DataProfiler.SQLCreator.ColumnProfingSQLCreate(estimate='sample', rowcount=100)

In [ ]:
### 샘플데이터 수집 SQL 생성
DataProfiler.SQLCreator.SampleProfingSQLCreate(estimate='sample', rowcount=100)

In [ ]:
### 오류 SQL에 대한 SQL 재생성
DataProfiler.SQLCreator.ExecuteErrorSQLCreate(estimate='sample', rowcount=100)

### 6. 프로파일링 SQL 수행
- degree : 동시에 수행할 프로세스 개수 지정
- duplicate : 리포지터리에 수집된 결과가 존재할 경우, 처리 방안 'merge'(지우고 입력), 'pass' (오류로 처리)
- echo(※미개발) : row(행별 출력), progress(진행바)

In [ ]:
DataProfiler.Executer.execute(degree=4, duplicate='merge')